# The Bibi Graph
## A social network analysis of leaders' autobiography <BR> via NER,language models, and graph algorithms
### Gal Engelberg <BR> January, 2023

# 1. Set up the neo4j environment & create the graph
Please assign the following variables:<br>
1. DB_URL for Neo4J bolt connection<br>
2. USER<br>
3. PASSWORD<br>
4. GRAPH_CREATE_CYPHER_PATH for create_graph.cypher path<br>

** You might need to increase the heap size in Neo4J configuration:<br>
1. dbms.memory.heap.initial_size=1G<br>
2. dbms.memory.heap.max_size=16G<br>

** Requirements:<br>
1. python packages: pandas, py2neo<br>
2. Neo4J instance version 4.4.11 Graph Data Science (GDS) Library version 2.1.12 

In [1]:
DB_URL = "bolt://localhost:7687"
USER = "neo4j"
PASSWORD = "qwerty"
GRAPH_CREATE_CYPHER_PATH = r"C:\Users\gal.engelberg\PycharmProjects\bibi_graph\Neo4j\Cypher folders\create_graph.cypher"

In [2]:
import pandas as pd
from py2neo import Graph,Node,Relationship
graph = Graph(DB_URL, auth=(USER, PASSWORD))

def run_query_from_file(file_name: str):
    try:
        query = ' '.join(
            open(file_name).readlines()).replace('\n', '')
        subqueries = query.split(';')
        results = []
        for subquery in subqueries:
            result = graph.run(subquery)
            results.append(result)
        print("done...")    
    except:
        print(e)
        
run_query_from_file(GRAPH_CREATE_CYPHER_PATH)

done...


Now, after the graph is up, and holds data, we create two sub-graph projections for the graph algorithms usage: <br>
1. Persons and events subgraph with the following objects: (PERSON)-[CO_OCCUR_WITH]-(EVENT) <br>
2. Persons, chapters, and events subgraph with the following objects: (PERSON:EVENT)-[MENTIONED_IN]->(CHAPTER)

In [3]:
# Create graph projections for graph analytics

# Persons-[co_occur_with]-events graph projection
persons_events = """CALL gds.graph.project(
  'persons_events',
  ['PERSON','EVENT'],
  'CO_OCCUR_WITH',
  {
    relationshipProperties: 'count'
  }
);"""

# Persons-chapters-events graph projection
persons_chapters_events = """CALL gds.graph.project(
  'persons_events_chapters',
  ['PERSON','EVENT','CHAPTER'],
  'MENTIONED_IN',
  {
    relationshipProperties: 'count'
  }
);"""

graph.run(persons_events)
graph.run(persons_chapters_events)

nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
"{PERSON: {label: 'PERSON', properties: {}}, CHAPTER: {label: 'CHAPTER', properties: {}}, EVENT: {label: 'EVENT', properties: {}}}","{MENTIONED_IN: {orientation: 'NATURAL', aggregation: 'DEFAULT', type: 'MENTIONED_IN', properties: {count: {defaultValue: null, property: 'count', aggregation: 'DEFAULT'}}}}",persons_events_chapters,770,1590,7


# 2. Graph Statistics

In [4]:
# Number of persons
query = "MATCH (n:PERSON) return count(distinct n) as number_of_persons"
print(graph.run(query))

 number_of_persons 
-------------------
               635 



In [5]:
# Number of Events
query = "MATCH (n:EVENT) return count(distinct n) as number_of_events"
print(graph.run(query))

 number_of_events 
------------------
               66 



In [6]:
# Number of MENTIONED_IN edges
query = "MATCH ()-[r:MENTIONED_IN]-() return count(distinct r) as number_of_mentioned_in_edges"
print(graph.run(query))

 number_of_mentioned_in_edges 
------------------------------
                         1590 



In [7]:
# Number of CO_OCCUR_WITH edges
query = "MATCH ()-[r:CO_OCCUR_WITH]-() return count(distinct r) as number_of_CO_OCCUR_WITH_edges"
print(graph.run(query))

 number_of_CO_OCCUR_WITH_edges 
-------------------------------
                         39146 



In [8]:
# Network diameter
query = """MATCH (n)
            WITH collect(n) AS nodes
            UNWIND nodes AS a
            UNWIND nodes AS b
            WITH a, b
            WHERE id(a) < id(b)
            MATCH path=shortestPath((a)-[*]-(b))
            RETURN length(path) AS diameter
            ORDER BY diameter
            DESC LIMIT 1"""
print(graph.run(query))

 diameter 
----------
        4 



In [9]:
# Degree distribution
query_degree = """CALL gds.degree.stream('persons_events')
            YIELD nodeId, score
            RETURN gds.util.asNode(nodeId).name AS name, score AS followers
            ORDER BY followers DESC, name DESC"""
df = graph.run(query_degree).to_data_frame()

query_weighted_degree = """CALL gds.degree.stream(
   'persons_events',
   { relationshipWeightProperty: 'count' }
)
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS name, score AS weightedFollowers
ORDER BY weightedFollowers DESC, name DESC"""

df1 = graph.run(query_weighted_degree).to_data_frame()

print(f"Degree Distribution: \n{df}")

print(f"Weighted Degree Distribution: \n{df1}")

# max degree
print(f"Max degree: {df['followers'].max()}")

# min degree
print(f"Min degree: {df['followers'].min()}")

# average degree
print(f"Average degree: {df['followers'].mean()}")

Degree Distribution: 
                    name  followers
0         Yoni Netanyahu      509.0
1            World War I      384.0
2           SHIMON PERES      383.0
3         Sara Netanyahu      374.0
4          Yitzhak Rabin      371.0
..                   ...        ...
696  Ithiel de Sola Pool        4.0
697          David Klein        4.0
698        Yitzhak Navon        0.0
699        Meir Har Zion        0.0
700            Danny Mat        0.0

[701 rows x 2 columns]
Weighted Degree Distribution: 
                    name  weightedFollowers
0         Yoni Netanyahu              941.0
1         Sara Netanyahu              720.0
2           SHIMON PERES              643.0
3          Yitzhak Rabin              625.0
4            World War I              612.0
..                   ...                ...
696  Ithiel de Sola Pool                4.0
697          David Klein                4.0
698        Yitzhak Navon                0.0
699        Meir Har Zion                0.0
700    

# 3. What are the cardinal persons according to the writer?

In [10]:
# Page rank
query = """CALL gds.pageRank.stream('persons_events', {
  maxIterations: 20,
  dampingFactor: 0.85,
  relationshipWeightProperty: 'count'
})
YIELD nodeId, score
with labels(gds.util.asNode(nodeId)) as label, gds.util.asNode(nodeId).name AS name, score
where 'PERSON' in label
RETURN name, score
ORDER BY score DESC, name ASC;"""
pr = graph.run(query).to_data_frame()

# Betweeness
query = """CALL gds.betweenness.stream('persons_events')
YIELD nodeId, score
with labels(gds.util.asNode(nodeId)) as label, gds.util.asNode(nodeId).name AS name, score
where 'PERSON' in label
RETURN name, score
ORDER BY score DESC, name ASC;"""
bt = graph.run(query).to_data_frame()

print(f"Cardinal persons (page rank): \n\n{pr}\n")

print(f"Cardinal persons (betweeness): \n\n{bt}\n")

Cardinal persons (page rank): 

               name      score
0    Yoni Netanyahu  11.926900
1    Sara Netanyahu   9.362623
2      SHIMON PERES   7.811885
3     Yitzhak Rabin   7.589419
4      Ariel Sharon   7.262936
..              ...        ...
630     David Klein   0.231362
631       Nir Gilad   0.231362
632       Danny Mat   0.150000
633   Meir Har Zion   0.150000
634   Yitzhak Navon   0.150000

[635 rows x 2 columns]

Cardinal persons (betweeness): 

                  name         score
0       Yoni Netanyahu  55002.528106
1       Sara Netanyahu  32617.093969
2         SHIMON PERES  20743.601200
3         Ariel Sharon  20346.539494
4    Benzion Netanyahu  18771.662053
..                 ...           ...
630        Zohar Linik      0.000000
631          Zvi Marom      0.000000
632  [Pinchas] Bukhris      0.000000
633         alte kaker      0.000000
634     the Vilna Gaon      0.000000

[635 rows x 2 columns]



# 4. What are the cardinal events according to the writer?

In [11]:
# Page rank
query = """CALL gds.pageRank.stream('persons_events', {
  maxIterations: 20,
  dampingFactor: 0.85,
  relationshipWeightProperty: 'count'
})
YIELD nodeId, score
with labels(gds.util.asNode(nodeId)) as label, gds.util.asNode(nodeId).name AS name, score
where 'EVENT' in label
RETURN name, score
ORDER BY score DESC, name ASC;"""
pr = graph.run(query).to_data_frame()

# Betweeness
query = """CALL gds.betweenness.stream('persons_events')
YIELD nodeId, score
with labels(gds.util.asNode(nodeId)) as label, gds.util.asNode(nodeId).name AS name, score
where 'EVENT' in label
RETURN name, score
ORDER BY score DESC, name ASC;"""
bt = graph.run(query).to_data_frame()

print(f"Cardinal events (page rank): \n\n{pr}\n")

print(f"Cardinal events (betweeness): \n\n{bt}\n")

Cardinal events (page rank): 

                         name     score
0                 World War I  7.600361
1                   Holocaust  6.860771
2             the Six-Day War  5.884391
3   War of Independence (1948  3.914277
4          The Yom Kippur War  3.573891
..                        ...       ...
61                Bar Mitzvah  0.340232
62    the Rothschild protests  0.325040
63       the Meal Tray Affair  0.319427
64            Black September  0.297298
65                  World Cup  0.253160

[66 rows x 2 columns]

Cardinal events (betweeness): 

                            name         score
0                    World War I  24196.605309
1                      Holocaust  21476.883705
2                the Six-Day War  15710.900859
3             The Yom Kippur War   7062.705208
4      War of Independence (1948   5586.188703
..                           ...           ...
61       the Spanish Inquisition      0.000000
62                the Tunnel War      0.000000
63      th

# 5. Can we discover meaningful communities in the graph?

Here, we extract communities from the graph using Louvain algorithm. We analyze the modularity of the network. Then We present each community with extra information as follows: # of entities within the community, % of total, average page rank of the entities within the community, and a list of top 25 entities within each community.

In [16]:
# Louvain
query_louvain = """CALL gds.louvain.stream('persons_events',{ relationshipWeightProperty: 'count' })
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).name AS name, communityId, intermediateCommunityIds
ORDER BY name ASC"""
com = graph.run(query_louvain).to_data_frame()

#retrieve modularity
mod = f"""CALL gds.louvain.write('persons_events', {{ writeProperty: 'community' }})
YIELD modularity"""
print(f"Modularity: \n\n {graph.run(mod)}")

query_page_rank = """CALL gds.pageRank.stream('persons_events', {
  maxIterations: 20,
  dampingFactor: 0.85,
  relationshipWeightProperty: 'count'
})
YIELD nodeId, score
with labels(gds.util.asNode(nodeId)) as label, gds.util.asNode(nodeId).name AS name, score
RETURN name, score
ORDER BY score DESC, name ASC;"""
pr = graph.run(query_page_rank).to_data_frame()

df = pd.merge(com, pr,on='name', how='inner')

# group by communityId and calculate average score
grouped = df.groupby('communityId').mean()

# create a new column with top 25 names by score
grouped['top25names'] = df.groupby('communityId').apply(lambda x: x.nlargest(25, 'score')['name'].tolist())

# add a column of count of names
grouped['name_count'] = df.groupby('communityId').count()['name']

# add a column of % of total names
grouped['percent_of_total'] = grouped['name_count'] / grouped['name_count'].sum()

# add average score of the whole group
grouped['average_score'] = df.groupby('communityId')['score'].mean()

# order by average_score
grouped.sort_values(by='average_score', ascending=False, inplace=True)

print(grouped)

Modularity: 

          modularity 
--------------------
 0.3850515589008596 

                score                                         top25names  \
communityId                                                                
51           1.587794  [Zeev Jabotinsky, Ben Gurion.1, War of Indepen...   
189          1.083686  [Yitzhak Rabin, Menachem Begin, Margaret Thatc...   
374          1.030944  [SHIMON PERES, Yasser Arafat, Ehud Barak, Bill...   
119          0.900183  [Benjamin Netanyahu, The Yom Kippur War, Yoav ...   
306          0.899823  [Sara Netanyahu, Benzion Netanyahu, Holocaust,...   
489          0.765688  [Yoni Netanyahu, World War I, the Six-Day War,...   
111          0.726566  [Ariel Sharon, Spencer Partrich, Second Intifa...   
98           0.401430  [Mahatma Gandhi, Deng Xiaoping, Jair Bolsonaro...   
197          0.150000                                    [Yitzhak Navon]   
364          0.150000                                        [Danny Mat]   
610      

# 6. Most “intensive” chapters

In [17]:
query = """CALL gds.pageRank.stream('persons_events_chapters', {
  maxIterations: 20,
  dampingFactor: 0.85,
  relationshipWeightProperty: 'count'
})
YIELD nodeId, score
with labels(gds.util.asNode(nodeId)) as label, gds.util.asNode(nodeId).name AS name, score
where 'CHAPTER' in label
RETURN name, score
ORDER BY score DESC, name ASC limit 10;"""

df = graph.run(query).to_data_frame()
print(f"Most intensive chapters: \n\n {df}")

Most intensive chapters: 

   name     score
0   20  5.464282
1   29  4.112944
2   28  4.110990
3   26  3.650599
4   38  3.599851
5   30  3.050861
6   32  2.675983
7   39  2.668428
8   19  2.450534
9   72  2.406484


# 7. Distribution of co occurrence between Yoni Netanyahu to other persons

In [18]:
query = """MATCH (n1:PERSON)-[r:CO_OCCUR_WITH]->(n2:PERSON)
        WHERE n1.name = 'Yoni Netanyahu' 
        return n2.name as name, r.count as number_of_co_occurence order by r.count desc"""

df = graph.run(query).to_data_frame()
print(f"Distribution of co occurrence between Yoni Netanyahu to other persons: \n\n {df}")

Distribution of co occurrence between Yoni Netanyahu to other persons: 

                   name  number_of_co_occurence
0       Iddo Netanyahu                      18
1    Benzion Netanyahu                      14
2         SHIMON PERES                      14
3        Yitzhak Rabin                      14
4       Sara Netanyahu                      11
..                 ...                     ...
461     Adolf Eichmann                       1
462        George Ball                       1
463         E. Housman                       1
464    Ruth Rubenstein                       1
465       Hanan Livneh                       1

[466 rows x 2 columns]


# 8. Persons similarity

In [19]:
query = """CALL gds.nodeSimilarity.stream('persons_events')
            YIELD node1, node2, similarity
            WITH gds.util.asNode(node1) as n1, gds.util.asNode(node2) as n2, similarity
            WHERE "PERSON" in labels(n1) and "PERSON" in labels(n2)
            UNWIND [[n1,n2],[n2,n1]] as x
            RETURN DISTINCT max(x[0].name) as name1, min(x[1].name) as name2, similarity
            ORDER BY similarity DESCENDING, name1, name2 limit 40"""
df = graph.run(query).to_data_frame()
print(f"Persons similarity: \n\n {df}")

Persons similarity: 

                    name1                     name2  similarity
0             Paul Nitze          Elmo Bud Zumwalt    0.979167
1            Peter Lubin             Elie Kedourie    0.977528
2           Moshe Katsav                David Levy    0.976744
3     Warren Christopher        Arthur Finkelstein    0.973333
4    William Ormsby-Gore    Arthur Schlesinger Jr.    0.972222
5          Jared Kushner            David Friedman    0.970588
6             Leah Rabin              Aaron Miller    0.967742
7         William Safire             Bernard Lewis    0.963636
8          Yelena Bonner              Ahmed Jibril    0.962963
9            Yossi Sarid       Alexander the Great    0.961538
10          Yossi Beilin            Alexander Zeid    0.960784
11  Yitzhak Itzik Molcho                  Al Buraq    0.959184
12      Vladimir Nabokov                 Aldo Moro    0.958333
13        Yaakov Amidror             Boyko Borisov    0.954545
14         Ronald Lauder        

# 9. Events similarity

In [20]:
query = """CALL gds.nodeSimilarity.stream('persons_events')
            YIELD node1, node2, similarity
            WITH gds.util.asNode(node1) as n1, gds.util.asNode(node2) as n2, similarity
            WHERE "EVENT" in labels(n1) and "EVENT" in labels(n2)
            UNWIND [[n1,n2],[n2,n1]] as x
            RETURN DISTINCT max(x[0].name) as name1, min(x[1].name) as name2, similarity
            ORDER BY similarity DESCENDING, name1, name2 limit 15"""
df = graph.run(query).to_data_frame()
print(f"Events similarity: \n\n {df}")

Events similarity: 

                                    name1                       name2  \
0                        post-Oslo flood         San Remo Conference   
1                     Wannsee Conference            Green Revolution   
2              Operation Northern Shield          Operation Guardian   
3           the Zionist Congress of 1905  the First Zionist Congress   
4                   The Syrian civil war                Lollapalooza   
5   Jerusalem Day, an annual celebration       Guardian of the Walls   
6                  the Arlosoroff Affair         San Remo Conference   
7                        post-Oslo flood                        Oslo   
8                        the Vietnam War     the Spanish Inquisition   
9                            World War I                   Holocaust   
10                 the Arlosoroff Affair     Madrid Peace Conference   
11                           World War I   War of Independence (1948   
12                       the Six-Day War  